In [1]:
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
import random as rand

In [39]:
# Question 2

# Read Video
video = cv.VideoCapture("ball.mov")

# Display Video
if (video.isOpened()== False): 
  print("Error opening video stream or file")

# Read until video is completed
while(video.isOpened()):
  # read frame-by-frame
  ret, frame = video.read()
  if ret == True:

    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    blur = cv.GaussianBlur(gray,(3,3),cv.BORDER_DEFAULT)

    circle = cv.HoughCircles(blur, cv.HOUGH_GRADIENT,1,20,param1=80,param2=20,minRadius=9,maxRadius=13) #plus minus 2 of radius to get better results

    if circle is not None:
        circle = np.int32(circle)
        for c in circle[0,:]:
            cv.circle(frame,(c[0],c[1]),c[2],(0,0,0),3)


    cv.imshow('Frame',frame)

 
    # Press Q on keyboard to  exit
    if cv.waitKey(25) & 0xFF == ord('q'):
      break

  else: 
    break

# When everything done, release the video videoture object
video.release()
 
# Closes all the frames
cv.destroyAllWindows()

# PRroblem 3

In [158]:
def solution3():
  # Read and resize the image
  tracks = cv.imread("train_track.jpg")
  tracks = cv.resize(tracks, (int(tracks.shape[1] * 0.3), int(tracks.shape[0] * 0.3)), interpolation=cv.INTER_AREA)
  
  # Filter the edges
  tracks_edges = cv.Canny(tracks, 625, 630)

  # Hough Transform
  lines = cv.HoughLines(tracks_edges, 1, np.pi/180, 225, None, 0, 0)
  
  # Extract rhos and thetas returned from HoughLines function to a list
  sorted_lines_polar = []
  if lines is not None:
    for i in range(0, len(lines)):
      sorted_lines_polar.append([lines[i][0][0], lines[i][0][1]])

  # Sort the list of lines in reverse order
  sorted_lines_polar.sort(reverse=True)

  # Pick two appropriate lines from the list
  for line in sorted_lines_polar:
    index = sorted_lines_polar.index(line)
    if index > 0 and abs(line[0] - sorted_lines_polar[index-1][0]) < 30:
      if line[0] > 0:
        sorted_lines_polar.remove(min(line, sorted_lines_polar[index-1]))
      if line[0] < 0:
        sorted_lines_polar.remove(max(line, sorted_lines_polar[index-1]))
  
  # Convert lines from polar form to two-point form for drawing in OpenCV
  def polar2twopoints(line):
    d = line[0]
    theta = line[1]
    cos_theta = np.cos(theta)
    sin_theta = np.sin(theta)
    x = d*cos_theta
    y = d*sin_theta
    x1 = int(x + 1500*(-sin_theta))
    y1 = int(y + 1500*(cos_theta))
    x2 = int(x - 1500*(-sin_theta))
    y2 = int(y - 1500*(cos_theta))
    return [x1, y1, x2, y2]
  
  sorted_lines_cartesian = []
  for line in sorted_lines_polar:
    x1, y1, x2, y2 = polar2twopoints(line)
    sorted_lines_cartesian.append([[x1, y1], [x2, y2]])
    cv.line(tracks, (x1, y1), (x2, y2), (0, 0, 255), 2)

  # Find the line equations and intersection from two-point form
  def getcoeffsandintersect(two_lines):
    coeffs_a_b = []
    coeffs_c = []
    for points in two_lines:
      x1, y1 = points[0]
      x2, y2 = points[1]
      a = round((y2 - y1)/(x2 - x1), 2)
      b = -1
      coeffs_a_b.append([a, b])
      c = round(a*x1 - y1, 2)
      coeffs_c.append([c])
    xc, yc = np.ravel(np.linalg.solve(coeffs_a_b, coeffs_c))
    return coeffs_a_b, coeffs_c, (round(xc), round(yc))
  
  A, B, intersection_point = getcoeffsandintersect(sorted_lines_cartesian)
  a1, b1 = A[0]
  c1 = B[0][0]
  a2, b2 = A[1]
  c2 = B[1][0]

  
  i_upper_left_corner_y = i_upper_right_corner_y = intersection_point[1] + 175

  i_upper_left_corner_x = round((c1 - b1*i_upper_left_corner_y)/a1)
  i_upper_right_corner_x = round((c2 - b2*i_upper_right_corner_y)/a2)

  i_upper_left_corner = (i_upper_left_corner_x, i_upper_left_corner_y)
  i_upper_right_corner = (i_upper_right_corner_x, i_upper_right_corner_y)

  i_lower_left_corner_y = i_lower_right_corner_y = tracks.shape[0]

  i_lower_left_corner_x = round((c1 - b1*i_lower_left_corner_y)/a1)
  i_lower_right_corner_x = round((c2 - b2*i_lower_right_corner_y)/a2)

  i_lower_left_corner = (i_lower_left_corner_x, i_lower_left_corner_y)
  i_lower_right_corner = (i_lower_right_corner_x, i_lower_right_corner_y)

  i_corners = np.float32([i_upper_left_corner, i_lower_left_corner, i_lower_right_corner, i_upper_right_corner])

  # Draw the corners
  cv.circle(tracks, intersection_point, 3, (0, 255, 0), -1)
  cv.circle(tracks, i_upper_left_corner, 3, (255, 128, 0), -1)
  cv.circle(tracks, i_upper_right_corner, 3, (0, 128, 255), -1)
  cv.circle(tracks, i_lower_left_corner, 3, (255, 128, 0), -1)
  cv.circle(tracks, i_lower_right_corner, 3, (0, 128, 255), -1)

  o_width = i_lower_right_corner_x - i_lower_left_corner_x
  o_height = round((np.sqrt((i_upper_right_corner_x - i_lower_right_corner_x)**2 + (i_upper_right_corner_y - i_lower_right_corner_y)**2))*(1/scale))
  o_size = (o_width, o_height)
  o_corners = np.float32([[0, 0], [0, o_size[1]-1], [o_size[0]-1, o_size[1]-1], [o_size[0]-1, 0]])

  matrix = cv.getPerspectiveTransform(i_corners, o_corners)

  tracks_topview = cv.warpPerspective(tracks, matrix, o_size, flags=cv.INTER_LINEAR)
  

  cv.imshow('Tracks', tracks)
  cv.imshow('Top view', tracks_topview)
  
  # Close windows upon pressing 'q'
  while True:
    cv.waitKey(0)
    cv.destroyAllWindows()
    break

In [166]:
solution3()

# Problem 4


In [140]:
baloon = cv.imread('hotairbaloon.jpg')
baloon = cv.resize(baloon, (int(baloon.shape[1] * 0.3), int(baloon.shape[0] * 0.3)), interpolation=cv.INTER_AREA)
gray_img= cv.cvtColor(baloon,cv.COLOR_BGR2GRAY)
blur_img= cv.GaussianBlur(gray_img,(7,7),0)
canny_img = cv.Canny(blur_img,150,200)
kernel = np.ones((1, 1), np.uint8)
erode = cv.erode(canny_img, kernel, iterations=1)
dilate = cv.dilate(erode, kernel, iterations=1)
contours,_= cv.findContours(dilate, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
cv.drawContours(image=baloon, contours=contours, contourIdx=-1, color=(0, 255, 0), thickness=2, lineType=cv.LINE_AA)
cv.imshow("baloon",baloon)
cv.waitKey()
cv.destroyAllWindows()

In [141]:
boundRect = []
contours_poly = []
for i, c in enumerate(contours):
    contours_poly.append(cv.approxPolyDP(c, 1, True))
    boundRect.append(cv.boundingRect(contours_poly[i]))

In [142]:
baloons_bound= []
for i in range(len(boundRect)):
    if boundRect[i][2]>50 and boundRect[i][3]>60:
        baloons_bound.append(boundRect[i])


In [143]:
drawing = np.zeros((canny_img.shape[0], canny_img.shape[1], 3), dtype=np.uint8)
for i in range(len(baloons_bound)):
    color = (rand.randint(0,256), rand.randint(0,256), rand.randint(0,256))
    cv.rectangle(baloon, (int(baloons_bound[i][0]), int(baloons_bound[i][1])),(int(baloons_bound[i][0]+baloons_bound[i][2]), int(baloons_bound[i][1]+baloons_bound[i][3])), color, 2)
cv.imshow("count",baloon)
cv.waitKey()
cv.destroyAllWindows()

In [144]:
len(baloons_bound)

20